In [ ]:
from PIL import Image
import torch
import matplotlib.pyplot as plt
import clip
import numpy as np

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = torch.hub.load('ultralytics/yolov5', 'yolov5s',pretrained=True)
clip_model , preprocess = clip.load('ViT-B/32',device)
model.to(device)

In [ ]:
img1 = Image.open('jack.jpg').convert("RGB")
crop_results = model(img1)
results = crop_results.crop(save=False)

In [ ]:
preprocessed_images = torch.stack([preprocess(Image.fromarray(result['im'])) for result in results])
search_query = "jack daniels"

In [ ]:
preprocessed_images = torch.tensor(np.stack(preprocessed_images)).cuda()
with torch.no_grad(): 
   images_features = clip_model.encode_image(preprocessed_images)
   text_encoded = clip_model.encode_text(clip.tokenize(search_query).to(device)) 
images_features /= images_features.norm(dim=-1, keepdim=True)
text_encoded /= text_encoded.norm(dim=-1, keepdim=True)

similarity = text_encoded.cpu().numpy() @ images_features.cpu().numpy().T

In [ ]:
def similarity_top(results,similarity_list,N):
  clip_results = zip(range(len(similarity_list)), similarity_list)
  clip_results = sorted(clip_results, key=lambda x: x[1],reverse= True)
  top_images = []
  scores=[]
  for index,score in clip_results[:N]:
    scores.append(score)
    top_images.append(results[index]['im'])
  return scores,top_images  

scores,imgs= similarity_top(results,similarity[0],N=1)

In [ ]:
xx = list(map(lambda x:int(x),results[similarity.argmax()]['box']))

In [ ]:
%matplotlib inline
plt.imshow(results[similarity.argmax()]['im'])
plt.show()

In [ ]:
img1.crop(xx)

In [ ]:
del similarity, preprocessed_images, search_query, results,crop_results, img1, images_features,imgs,scores,text_encoded
torch.cuda.empty_cache()